In [1]:
#  데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer
# 행렬 등 통계 연산 모듈
import numpy as np
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations

# 데이터 시각화
import matplotlib.pyplot as plt
# from matplotlib.pyplot import
import matplotlib.font_manager as fm
import missingno as msno
import seaborn as sns

Mounted at /content/drive


In [2]:
# apt-get : 패키지 설치 명령어
# -qq : 에러외의 메세지 숨기는 옵션키
!apt-get -qq install fonts-nanum

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
    # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')
# 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)
# Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})
# 폰트 설정

Selecting previously unselected package fonts-nanum.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [16]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    self.data = pd.concat(self.data)

    return self.data


In [17]:
mountInstance = LoadGoogleDriveData()

### 1.오아시스 비즈니스(필지고유번호 & 법정동읍면동코드 & 외식업종)  데이터

In [5]:
CONTEST_2023 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "소상공인 매출등급 예상 대상 필지.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

In [6]:
CONTEST_2022 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "필지단위 소상공인 매출등급 정보.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

### 2. KCD(상권코드 & 외식업종 중분류) 소상공인 신용데이터

In [7]:
KCD_2022_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "SOHO_MERGE_DATA_LIST.csv",
    unicode = "utf-8", columnTF = False)

### 3. 행정동코드 - 상권코드 매핑 데이터

In [8]:
MARKET_LOC_MAP = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/",
    file_name_extension = "market_loc_map.csv",
    unicode = "euc-kr", columnTF = False)

In [9]:
MARKET_LOC_MAP

,상권_코드_2022,상권_코드_2023,행정동_코드,법정동_코드,STDG_EMD_CD
0,2110008,3110008,11110515,11110101,11110101
1,2110008,3110008,11110515,11110102,11110102
2,2110008,3110008,11110515,11110103,11110103
3,2110008,3110008,11110515,11110104,11110104
4,2110008,3110008,11110515,11110105,11110105
...,...,...,...,...,...
3453,2130102,3130102,11230750,11230110,11230110
3454,2130166,3130166,11380590,11380107,11380107
3455,2130113,3130113,11260690,11260106,11260106
3456,2130124,3130124,11290780,11290138,11290138


### 4. 서울열린데이터광장 상권특성 공공데이터

In [18]:
MERGE_2022_DATA = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_MERGE_2022_RAW_DATA.csv",
    unicode = "utf-8", chunking_row_num = 10**5, columnTF = False)

In [19]:
MERGE_2022_DATA

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명_x,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,...,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,상권_코드_명_y,엑스좌표_값,와이좌표_값,시군구_코드
0,2022,4,A,골목상권,2110001,이북5도청사,4305934.0000,8.0000,520427635.0000,129126028.0000,...,HH,정체,131,63,123,56,평창동서측,197064,456643,11110
1,2022,4,A,골목상권,2110001,이북5도청사,4305934.0000,8.0000,520427635.0000,129126028.0000,...,HH,정체,131,63,123,56,이북5도청사,196262,456660,11110
2,2022,4,A,골목상권,2110001,이북5도청사,4305934.0000,8.0000,520427635.0000,129126028.0000,...,HH,정체,131,63,123,56,평창동동측,197827,457040,11110
3,2022,4,A,골목상권,2110001,이북5도청사,4305934.0000,8.0000,520427635.0000,129126028.0000,...,HH,정체,131,63,123,56,평창동서측,197064,456643,11110
4,2022,4,A,골목상권,2110001,이북5도청사,4305934.0000,8.0000,520427635.0000,129126028.0000,...,HH,정체,131,63,123,56,이북5도청사,196262,456660,11110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188036,2022,1,A,골목상권,2110793,대림썬프라자,NaN,NaN,NaN,NaN,...,HL,상권축소,127,52,119,55,대림한신아파트,191471,443882,11560
2188037,2022,1,A,골목상권,2110793,대림썬프라자,NaN,NaN,NaN,NaN,...,HL,상권축소,127,52,119,55,대림썬프라자,191840,444066,11560
2188038,2022,1,A,골목상권,2110793,대림썬프라자,NaN,NaN,NaN,NaN,...,HL,상권축소,127,52,119,55,대림갑을명가아파트,191416,444092,11560
2188039,2022,1,A,골목상권,2110793,대림썬프라자,NaN,NaN,NaN,NaN,...,HL,상권축소,127,52,119,55,신대림초등학교,191504,443609,11560


In [20]:
MERGE_2023_DATA = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_MERGE_2023_RAW_DATA.csv",
    unicode = "utf-8", chunking_row_num = 10**5, columnTF = False)

<ipython-input-16-8871afbc47ee>:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = list(chunkdata)
<ipython-input-16-8871afbc47ee>:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = list(chunkdata)
<ipython-input-16-8871afbc47ee>:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = list(chunkdata)


In [21]:
MERGE_2023_DATA

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명_x,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,...,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,상권_코드_명_y,엑스좌표_값,와이좌표_값,시군구_코드
0,2023,1,A,골목상권,3110001.0000,이북5도청사,NaN,NaN,520427635.0000,129126028.0000,...,HH,정체,111.0000,58.0000,104.0000,52.0000,평창동서측,197064.0000,456643.0000,11110.0000
1,2023,1,A,골목상권,3110001.0000,이북5도청사,NaN,NaN,520427635.0000,129126028.0000,...,HH,정체,111.0000,58.0000,104.0000,52.0000,이북5도청사,196262.0000,456660.0000,11110.0000
2,2023,1,A,골목상권,3110001.0000,이북5도청사,NaN,NaN,520427635.0000,129126028.0000,...,HH,정체,111.0000,58.0000,104.0000,52.0000,평창동동측,197827.0000,457040.0000,11110.0000
3,2023,1,A,골목상권,3110001.0000,이북5도청사,NaN,NaN,520427635.0000,129126028.0000,...,HH,정체,111.0000,58.0000,104.0000,52.0000,평창동서측,197064.0000,456643.0000,11110.0000
4,2023,1,A,골목상권,3110001.0000,이북5도청사,NaN,NaN,520427635.0000,129126028.0000,...,HH,정체,111.0000,58.0000,104.0000,52.0000,이북5도청사,196262.0000,456660.0000,11110.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575900,2023,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,LL,다이나믹,87.0000,42.0000,104.0000,52.0000,NaN,NaN,NaN,NaN
575901,2023,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,HH,정체,110.0000,52.0000,104.0000,52.0000,NaN,NaN,NaN,NaN
575902,2023,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,HH,정체,106.0000,57.0000,104.0000,52.0000,NaN,NaN,NaN,NaN
575903,2023,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,HH,정체,120.0000,53.0000,104.0000,52.0000,NaN,NaN,NaN,NaN


In [ ]:
"""
# 텍스트 파일 n개가 포함된 C(로컬디스크) 내 파일경로 설정
file_path = "/content/drive/MyDrive/2023BigContest/data" + "/" + "상권_2022"
# 파일경로 내 텍스트 파일명을 디렉토리 리스트에 저장
directory_상권_2022 = os.listdir(file_path)
print(directory_상권_2022, end = ", ")

data_상권특성_2022: List = [
    (directory_상권_2022[index].split("(")[1]).split(")")[0]
    for index in range(len(directory_상권_2022))
]
print("\n", data_상권특성_2022, end = ", ")

# 텍스트 파일 n개가 포함된 C(로컬디스크) 내 파일경로 설정
file_path = "/content/drive/MyDrive/2023BigContest/data" + "/" + "상권_2023"
# 파일경로 내 텍스트 파일명을 디렉토리 리스트에 저장
directory_상권_2023 = os.listdir(file_path)
print(directory_상권_2023, end = ",")

data_상권특성_2023: List = [
    (directory_상권_2023[index].split("(")[1]).split(")")[0]
    for index in range(len(directory_상권_2023))
]
print("\n", data_상권특성_2023, end = ", ")

# 상권특성_2022 및 상권특성_2023 데이터의 교집합
print(np.intersect1d(data_상권특성_2022, data_상권특성_2023))
# 상권특성_2022 및 상권특성_2023 데이터의 여집합
print(np.setxor1d(data_상권특성_2022, data_상권특성_2023))
directory_상권_2022.remove("서울시 상권분석서비스(상권배후지-소득소비).csv")
data_상권특성_2022.remove("상권배후지-소득소비")

MARKET_2022_LIST = []
for index, data in zip(list(range(len(directory_상권_2022))), directory_상권_2022):
  MARKET_2022 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/상권_2022/",
    file_name_extension = data,
    unicode = "euc-kr", columnTF = False)
  MARKET_2022_LIST.append(MARKET_2022)
  MARKET_2022 = 0
MARKET_2023_LIST = []
directory_상권_2023: List = [item.split(".csv")[0] + "_2023.csv" for item in directory_상권_2022[1:]]

for index, data in zip(list(range(len(directory_상권_2023))), directory_상권_2023):
  MARKET_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/상권_2023/",
    file_name_extension = data,
    unicode = "euc-kr", columnTF = False)
  MARKET_2023_LIST.append(MARKET_2023)
  MARKET_2023 = 0

display(directory_상권_2022)
# MARKET_2022_LIST[0]
"""

### 5.분기별 상권특성 공공데이터 전처리

In [22]:
MERGE_2022_DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2188041 entries, 0 to 2188040
Data columns (total 92 columns):
 #   Column              Dtype  
---  ------              -----  
 0   기준_년_코드             int64  
 1   기준_분기_코드            int64  
 2   상권_구분_코드            object 
 3   상권_구분_코드_명          object 
 4   상권_코드               int64  
 5   상권_코드_명_x           object 
 6   월_평균_소득_금액          float64
 7   소득_구간_코드            float64
 8   지출_총금액              float64
 9   식료품_지출_총금액          float64
 10  의류_신발_지출_총금액        float64
 11  생활용품_지출_총금액         float64
 12  의료비_지출_총금액          float64
 13  교통_지출_총금액           float64
 14  여가_지출_총금액           float64
 15  문화_지출_총금액           float64
 16  교육_지출_총금액           float64
 17  유흥_지출_총금액           float64
 18  아파트_단지_수            float64
 19  아파트_가격_1_억_미만_세대_수  float64
 20  아파트_가격_1_억_세대_수     float64
 21  아파트_가격_2_억_세대_수     float64
 22  아파트_가격_3_억_세대_수     float64
 23  아파트_가격_4_억_세대_수     float64
 24  아파트_가격_5_억_세대_수     floa

#### 5.(1) 상권_소득소비

  MARKET_2022_LIST[2]

  MARKET_2023_LIST[1]

  ```
  SELECT
    AVERAGE(seoul.상권특성.교육_지출_총금액),
    AVERAGE(seoul.상권특성.교통_지출_총금액),
    AVERAGE(seoul.상권특성.생활용품_지출_총금액),
    AVERAGE(seoul.상권특성.문화_지출_총금액),
    AVERAGE(seoul.상권특성.유흥_지출_총금액),
    AVERAGE(seoul.상권특성.의료비_지출_총금액),
    AVERAGE(seoul.상권특성.의류_신발_지출_총금액),
    AVERAGE(seoul.상권특성.식료품_지출_총금액),
    AVERAGE(seoul.상권특성.여가_지출_총금액),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_no = seoul.상권특성.상권_코드
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [23]:
MERGE_2022_DATA["상권_구분_코드_명"].unique()

array(['골목상권', '발달상권', '전통시장'], dtype=object)

In [26]:
MERGE_2022_DATA["서비스_업종_코드_명"].unique()
# 외식업종만 찾아서 행 추출

array(['전자상거래업', '인테리어', '철물점', '가전제품', '섬유제품', '운동/경기용품', '일반의류', '육류판매',
       '편의점', '슈퍼마켓', '통번역서비스', '사진관', '노래방', '여관', '부동산중개업', '미용실',
       '골프연습장', '당구장', '한의원', '일반교습학원', '커피-음료', '호프-간이주점', '분식전문점',
       '일식음식점', '한식음식점', '조명용품', '애완동물', '화장품', '서적', '의료기기', '의약품',
       '반찬가게', '청과상', '핸드폰', '독서실', '세탁소', '피부관리실', '모터사이클수리', '자동차수리',
       'PC방', '치과의원', '예술학원', '외국어학원', '치킨전문점', '제과점', '양식음식점', '예술품',
       '가구', '화초', '미곡판매', '주류도매', '여행사', '네일숍', '스포츠클럽', '기타오락장', '일반의원',
       '스포츠 강습', '시계및귀금속', '가전제품수리', '패스트푸드점', '컴퓨터및주변장치판매', '세무사사무소',
       '가방', '주유소', '자동차부품', '게스트하우스', '문구', '안경', '고시원', '자동차미용',
       '법무사사무소', '중식음식점', 'DVD방', '신발', '재생용품 판매점', '악기', '수산물판매', '의류임대',
       '건축물청소', '녹음실', '기타법무서비스', '변호사사무소', '미용재료', '한복점', '통신기기수리',
       '볼링장', '회계사사무소', '비디오/서적임대', '컴퓨터학원', '모터사이클및부품', '유아의류', '복권방',
       '완구', '동물병원', '자전거 및 기타운송장비', '중고가구', '가정용품임대', '중고차판매', '전자게임장',
       '변리사사무소', nan], dtype=object)

In [27]:
# MERGE_2022_DATA.groupby(["상권_코드", "상권_코드_명_x"])[["지출_총금액", "식료품_지출_총금액"]].agg(lambda x: x.std() / x.mean())

,,지출_총금액,식료품_지출_총금액
상권_코드,상권_코드_명_x,,
2110001,이북5도청사,0.0000,0.0000
2110002,독립문역 1번,0.0000,0.0000
2110003,세검정초등학교,0.0000,0.0000
2110004,대신고등학교,0.0000,0.0000
2110005,세검정,0.0000,0.0000
...,...,...,...
2130322,고분다리전통시장,0.0000,0.0000
2130323,둔촌역전통시장,0.0000,0.0000
2130324,길동복조리시장,0.0000,0.0000


#### 5.(2) 상권_배후지_아파트
  
  MARKET_2022_LIST[3]

  MARKET_2023_LIST[2]

  ```
  SELECT
    AVERAGE(seoul.상권특성.아파트_단지_수),
    AVERAGE(seoul.상권특성.아파트_가격_1_억_미만_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_1_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_2_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_3_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_4_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_5_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_6_억_이상_세대_수),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_nm = seoul.상권특성.상권_코드_명
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

#### 5.(3) 상권배후지_점포

  MARKET_2022_LIST[6]

  MARKET_2023_LIST[5]

  ```
  SELECT
    AVERAGE(seoul.상권특성.개업_점포_수),
    AVERAGE(seoul.상권특성.유사_업종_점포_수),
    AVERAGE(seoul.상권특성.폐업_점포_수),
    AVERAGE(seoul.상권특성.점포_수),
    AVERAGE(seoul.상권특성.프랜차이즈_점포_수),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.개업_율),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.폐업_율),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_nm = seoul.상권특성.상권_코드_명
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

#### 5.(4) 상권배후지_집객시설

  MARKET_2022_LIST[7]

  MARKET_2023_LIST[6]
    
  ```
  SELECT
    AVERAGE(seoul.상권특성.집객시설_수),
    AVERAGE(seoul.상권특성.관공서_수),
    AVERAGE(seoul.상권특성.은행_수),
    AVERAGE(seoul.상권특성.종합병원_수),
    AVERAGE(seoul.상권특성.일반_병원_수),
    AVERAGE(seoul.상권특성.약국_수),
    AVERAGE(seoul.상권특성.유치원_수),
    AVERAGE(seoul.상권특성.초등학교_수),
    AVERAGE(seoul.상권특성.고등학교_수),
    AVERAGE(seoul.상권특성.대학교_수),
    AVERAGE(seoul.상권특성.백화점_수),
    AVERAGE(seoul.상권특성.슈퍼마켓_수),
    AVERAGE(seoul.상권특성.숙박_시설_수),AVERAGE(seoul.상권특성.버스_터미널_수),
    AVERAGE(seoul.상권특성.지하철_역_수),
    AVERAGE(seoul.상권특성.버스_정거장_수),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_no = seoul.상권특성.상권_코드
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

#### 5.(5) 상권_상주인구(상권-생활인구)

  MARKET_2022_LIST[11]

  MARKET_2022_LIST[10]

  ```
  SELECT
    AVERAGE(seoul.상권특성.연령대_10_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_20_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_30_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_40_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_50_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_60_이상_상주인구_수),
    AVERAGE(seoul.상권특성.총_가구_수),
    AVERAGE(seoul.상권특성.비_아파트_가구_수),
    AVERAGE(seoul.상권특성.아파트_가구_수),
    AVERAGE(seoul.상권특성.총_상주인구_수),
    AVERAGE(seoul.상권특성.남성_상주인구_수),
    AVERAGE(seoul.상권특성.여성_상주인구_수),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_no = seoul.상권특성.상권_코드
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [23]:
# MARKET_2022_LIST[11].rename(columns = {" 상권_구분_코드_명":"상권_구분_코드_명"}, inplace = True)
# MARKET_2023_LIST[10].rename(columns = {" 상권_구분_코드_명":"상권_구분_코드_명"}, inplace = True)

#### 5.(6) 상권_점포(행정동별 상권변화지표)

  ['상권축소', '정체', '다이나믹', '상권확장']
  
  MARKET_2022_LIST[13]

  MARKET_2023_LIST[12]

  ```
  SELECT
    AVERAGE(seoul.상권특성.개업_점포_수),
    AVERAGE(seoul.상권특성.유사_업종_점포_수),
    AVERAGE(seoul.상권특성.폐업_점포_수),
    AVERAGE(seoul.상권특성.점포_수),
    AVERAGE(seoul.상권특성.프랜차이즈_점포_수),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.개업_율),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.폐업_율),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_nm = seoul.상권특성.상권_코드_명
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

___

In [ ]:
"""
for index in [2, 3, 6, 7, 11]:
  if index == 2:
    MERGE_2022_DATA = MARKET_2022_LIST[index]
  else:
    MERGE_2022_DATA = pd.merge(
        left = MERGE_2022_DATA, right = MARKET_2022_LIST[index],
        on = ["기준_년_코드", "기준_분기_코드", "상권_구분_코드", "상권_구분_코드_명", "상권_코드", "상권_코드_명"],
        how = "outer")
MARKET_2022_LIST[0].drop(["형태정보"], axis = 1, inplace = True)
MARKET_LOC_MAP.info()
MARKET_2022_LIST[13] = pd.merge(
    left = MARKET_LOC_MAP, right = MARKET_2022_LIST[13],
    on = ["행정동_코드"],
    how = "right")
MARKET_2022_LIST[13].rename(columns = {"상권_코드_2022":"상권_코드"}, inplace = True)
MERGE_2022_DATA = pd.merge(
        left = MERGE_2022_DATA, right = MARKET_2022_LIST[13],
        on = ["기준_년_코드", "기준_분기_코드", "상권_코드"],
        how = "inner")
MERGE_2022_DATA = pd.merge(
        left = MERGE_2022_DATA, right = MARKET_2022_LIST[0][["상권_구분_코드", "상권_코드_명", "엑스좌표_값", "와이좌표_값", "시군구_코드", "행정동_코드"]],
        on = ["상권_구분_코드", "행정동_코드"],
        how = "left")
pd.set_option("display.max_column", None)
MERGE_2022_DATA
MERGE_2022_DATA.info()
MERGE_2022_DATA.to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/MARKET_MERGE_2022_RAW_DATA.csv", index = False)
"""

___

In [ ]:
"""
for index in [1, 2, 5, 6, 10]:
  if index == 1:
    MERGE_2023_DATA = MARKET_2023_LIST[index]
  else:
    MERGE_2023_DATA = pd.merge(
        left = MERGE_2023_DATA, right = MARKET_2023_LIST[index],
        on = ["기준_년_코드", "기준_분기_코드", "상권_구분_코드", "상권_구분_코드_명", "상권_코드", "상권_코드_명"],
        how = "outer")
MARKET_2023_LIST[10]
MERGE_2023_DATA.info()
MARKET_LOC_MAP.info()
MARKET_2023_LIST[12] = pd.merge(
    left = MARKET_LOC_MAP, right = MARKET_2023_LIST[12],
    on = ["행정동_코드"],
    how = "inner")
MARKET_2023_LIST[12].rename(columns = {"상권_코드_2023":"상권_코드"}, inplace = True)
MERGE_2023_DATA.info()
MARKET_2023_LIST[12].info()
MERGE_2023_DATA = pd.merge(
        left = MERGE_2023_DATA, right = MARKET_2023_LIST[12],
        on = ["기준_년_코드", "기준_분기_코드", "상권_코드"],
        how = "outer")
MERGE_2023_DATA = pd.merge(
        left = MERGE_2023_DATA, right = MARKET_2022_LIST[0][["상권_구분_코드", "상권_코드_명", "엑스좌표_값", "와이좌표_값", "시군구_코드", "행정동_코드"]],
        on = ["상권_구분_코드", "행정동_코드"],
        how = "left")
MERGE_2023_DATA.to_csv("/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/MARKET_MERGE_2023_RAW_DATA.csv", index = False)
"""